name: new_dataset_tests_default_params.ipynb

This notebook is very similar to new_dataset_tests.ipynb, but runs slightly different experiments.

Please note that this notebook is not set up with the current "experiments" folder as the destination for results. I would not recommend running this, please use `MLME-v2.0-conv_model.py` instead. 

---

https://github.com/tensorflow/tensorflow/issues/38012 for keras no module error

In [ ]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/Shareddrives/NRC_Amii_Agronomics_Project/nrc-ml-plant-genomics/'

In [ ]:
import keras
import warnings, logging
import json
import numpy as np
import pandas as pd
import datetime, os

from models.base_model import BaseModel
from keras.models import Sequential, load_model, model_from_json
from keras.layers import Input, Dense, Conv1D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping  # https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
import tensorflow as tf

from sklearn.metrics import r2_score
from scipy.stats import spearmanr # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html

warnings.filterwarnings('ignore')
logging.disable(1000)

%load_ext tensorboard

In [ ]:
ls

LICENSE                                 linear_mapping.ipynb
README.md                               logs/
__init__.py                             main.py
commands_for_setup.txt                  models/
create_final_dataset.ipynb              new_data/
cross_val_linear_mapping.ipynb          new_dataset_tests.ipynb
data/                                   new_dataset_tests_default_params.ipynb
data_exploration.ipynb                  new_dataset_tests_duplicate.ipynb
data_loader/                            new_dataset_tests_more_epochs.ipynb
evaluator/                              new_models/
example/                                new_scripts/
generate_data_format.ipynb              predict_on_NRC_data.ipynb
keras_model_loading.ipynb               requirements.txt
keras_retraining.ipynb                  requirements_exact.txt
keras_training.ipynb                    trainers/
kipoi/                                  utils/
kipoi_playground.ipynb


In [ ]:
# params

learning_rate_ = 0.002
batch_size_ = 512
num_epochs_ = 100
patience_ = 20

In [ ]:
# load in model as it is from kipoi

with open('models/model.json', 'r') as json_file:
    json_savedModel = json_file.read()
    
pretrained_model = model_from_json(json_savedModel)
pretrained_model.load_weights('models/pretrained.hdf5')

In [ ]:
pretrained_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batch_normalization_1 (Batch (None, 141, 120)          480       
_________________________________________________________________
dropout_1 (Dropout)          (None, 141, 120)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batch_normalization_2 (Batch (None, 137, 120)          480       
_________________________________________________________________
dropout_2 (Dropout)          (None, 137, 120)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 133, 120)         

In [ ]:
mapping = {"A":[1, 0, 0, 0], "T":[0, 0, 0, 1], "C":[0, 1, 0, 0], "G":[0, 0, 1, 0]}  # cross referenced with kipoi data loader

def get_ohe(sequence):  # gets sequence in format model can use
    return np.array([mapping[nt] for nt in sequence])

def get_model(treatment_dict):  # initializes model architecture
    mdl = Sequential()

    conv1_train = treatment_dict["conv1"] != 2  # bool
    mdl.add(Conv1D(120, 5, activation='relu', input_shape=(145, 4), name="1DConv_1", trainable=conv1_train))
    mdl.add(BatchNormalization(name="batchNorm1", trainable=conv1_train))
    mdl.add(Dropout(0.1, name="drop1"))

    conv2_train = treatment_dict["conv2"] != 2  # bool 
    mdl.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=conv2_train))
    mdl.add(BatchNormalization(name="batchNorm2", trainable=conv2_train))
    mdl.add(Dropout(0.1, name="drop2"))

    conv3_train = treatment_dict["conv3"] != 2  # bool 
    mdl.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=conv3_train))
    mdl.add(BatchNormalization(name="batchNorm3", trainable=conv3_train))
    mdl.add(Dropout(0.1, name="drop3"))

    mdl.add(Flatten(name="flat"))    
    mdl.add(Dense(1, activation='linear', name="dense2"))
    
    return mdl

def set_weights(treatment_dict, mdl):  # sets appropriate model weights from pretrained
    layers_to_set = []
    if treatment_dict["conv1"] != 0: layers_to_set += [0, 1, 2]
    if treatment_dict["conv2"] != 0: layers_to_set += [3, 4, 5]
    if treatment_dict["conv3"] != 0: layers_to_set += [6, 7, 8]
    
    for i in layers_to_set:
        print(mdl.layers[i])
        pretrained_layer_weights = pretrained_model.layers[i].get_weights()
        mdl.layers[i].set_weights(pretrained_layer_weights)
    
    return mdl

# NOT USED
def return_spearman(true, pred):
    return tf.convert_to_tensor(spearmanr(true, pred)[0])

# NOT USED
def correlation(x, y):  # https://www.kaggle.com/carlolepelaars/understanding-the-metric-spearman-s-rho
    mx = tf.math.reduce_mean(x)
    my = tf.math.reduce_mean(y)
    xm, ym = x-mx, y-my
    r_num = tf.math.reduce_mean(tf.multiply(xm,ym))        
    r_den = tf.math.reduce_std(xm) * tf.math.reduce_std(ym)
    return r_num / r_den

In [ ]:
# loading in data
df = pd.read_csv("data/processed/hidra_chloroplast.csv")

In [ ]:
df.head()

,organelle,start_coords,end_coords,sequence,control_raw_coverage,treatment_raw_coverage,control_norm_coverage,treatment_norm_coverage,target,A,T,C,G,set
0,NC_016734.1,0,145,AATCATAATAACTTGGTCCCGGGCATCACGGGCGAACGACGGGAAT...,998,316,338.31,405.52,0.26,39,40,40,26,test
1,NC_016734.1,5,150,TAATAACTTGGTCCCGGGCATCACGGGCGAACGACGGGAATTGAAC...,998,318,338.31,408.08,0.27,40,40,39,26,test
2,NC_016734.1,10,155,ACTTGGTCCCGGGCATCACGGGCGAACGACGGGAATTGAACCCGCG...,998,318,338.31,408.08,0.27,38,38,42,27,test
3,NC_016734.1,15,160,GTCCCGGGCATCACGGGCGAACGACGGGAATTGAACCCGCGATGGT...,998,318,338.31,408.08,0.27,40,36,42,27,test
4,NC_016734.1,20,165,GGGCATCACGGGCGAACGACGGGAATTGAACCCGCGATGGTGAATT...,998,318,338.31,408.08,0.27,41,39,39,26,test


In [ ]:
# split into different datasets

train_df = df[df.set=="train"]
X_train = np.array([get_ohe(sqnc) for sqnc in train_df["sequence"]])
y_train = np.array(train_df["target"].tolist())

val_df = df[df.set=="val"]
X_val = np.array([get_ohe(sqnc) for sqnc in val_df["sequence"]])
y_val = np.array(val_df["target"].tolist())

test_df = df[df.set=="test"]
X_test = np.array([get_ohe(sqnc) for sqnc in test_df["sequence"]])
y_test = np.array(test_df["target"].tolist())

In [ ]:
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(23804, 145, 4) (2964, 145, 4) (2964, 145, 4)
(23804,) (2964,) (2964,)


### Re-train from scratch

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":0, 
                   "conv2":0, 
                   "conv3":0}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          7

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate_),  # https://stackoverflow.com/questions/59737875/keras-change-learning-rate
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=patience_)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=num_epochs_, 
                    batch_size=batch_size_,
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
47/47 [==============================] - 44s 668ms/step - loss: 1.9685 - val_loss: 0.2745
Epoch 2/100
47/47 [==============================] - 31s 656ms/step - loss: 0.7293 - val_loss: 0.3983
Epoch 3/100
47/47 [==============================] - 37s 795ms/step - loss: 0.4306 - val_loss: 0.2738
Epoch 4/100
47/47 [==============================] - 33s 697ms/step - loss: 0.3210 - val_loss: 0.5855
Epoch 5/100
47/47 [==============================] - 31s 651ms/step - loss: 0.2483 - val_loss: 0.4949
Epoch 6/100
47/47 [==============================] - 30s 639ms/step - loss: 0.2191 - val_loss: 0.5036
Epoch 7/100
47/47 [==============================] - 31s 662ms/step - loss: 0.1827 - val_loss: 1.0355
Epoch 8/100
47/47 [==============================] - 30s 638ms/step - loss: 0.1506 - val_loss: 0.7021
Epoch 9/100
47/47 [==============================] - 30s 629ms/step - loss: 0.1247 - val_loss: 0.4279
Epoch 10/100
47/47 [==============================] - 29s 624ms/step - loss: 0.109

In [ ]:
%tensorboard --logdir logs

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.9875753653737224
Val 0.2688763874965914
Test 0.0925479516504516


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.9975163568931519, pvalue=0.0)
Val SpearmanrResult(correlation=0.5455477573814874, pvalue=1.5852960595821056e-229)
Test SpearmanrResult(correlation=0.30404121759112374, pvalue=1.969640894672249e-64)


### Re-train from starting point

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":1, 
                   "conv2":1, 
                   "conv3":1}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)         

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate_),  # https://stackoverflow.com/questions/59737875/keras-change-learning-rate
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=patience_)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=num_epochs_, 
                    batch_size=batch_size_,
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
47/47 [==============================] - 33s 679ms/step - loss: 0.2047 - val_loss: 0.3154
Epoch 2/100
47/47 [==============================] - 31s 664ms/step - loss: 0.0751 - val_loss: 0.2628
Epoch 3/100
47/47 [==============================] - 30s 648ms/step - loss: 0.0264 - val_loss: 0.2433
Epoch 4/100
47/47 [==============================] - 30s 636ms/step - loss: 0.0188 - val_loss: 0.2090
Epoch 5/100
47/47 [==============================] - 29s 610ms/step - loss: 0.0152 - val_loss: 0.2083
Epoch 6/100
47/47 [==============================] - 28s 605ms/step - loss: 0.0136 - val_loss: 0.1910
Epoch 7/100
47/47 [==============================] - 31s 652ms/step - loss: 0.0117 - val_loss: 0.1949
Epoch 8/100
47/47 [==============================] - 30s 635ms/step - loss: 0.0107 - val_loss: 0.1848
Epoch 9/100
47/47 [==============================] - 28s 591ms/step - loss: 0.0097 - val_loss: 0.1840
Epoch 10/100
47/47 [==============================] - 28s 598ms/step - loss: 0.008

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 84889), started 0:18:21 ago. (Use '!kill 84889' to kill it.)

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.9849067843119053
Val 0.29334543590715645
Test 0.13985236707904303


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.9964140966742986, pvalue=0.0)
Val SpearmanrResult(correlation=0.5593057890658346, pvalue=1.3085532306592987e-243)
Test SpearmanrResult(correlation=0.3803089308672248, pvalue=1.2633850016353279e-102)


### Train only output layer

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":2, 
                   "conv2":2, 
                   "conv3":2}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)         

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate_),  # https://stackoverflow.com/questions/59737875/keras-change-learning-rate
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=patience_)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=num_epochs_, 
                    batch_size=batch_size_,
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
47/47 [==============================] - 10s 190ms/step - loss: 0.2246 - val_loss: 0.1665
Epoch 2/100
47/47 [==============================] - 9s 189ms/step - loss: 0.1880 - val_loss: 0.1653
Epoch 3/100
47/47 [==============================] - 9s 185ms/step - loss: 0.1839 - val_loss: 0.1644
Epoch 4/100
47/47 [==============================] - 8s 172ms/step - loss: 0.1785 - val_loss: 0.1623
Epoch 5/100
47/47 [==============================] - 8s 171ms/step - loss: 0.1727 - val_loss: 0.1617
Epoch 6/100
47/47 [==============================] - 8s 171ms/step - loss: 0.1688 - val_loss: 0.1647
Epoch 7/100
47/47 [==============================] - 8s 167ms/step - loss: 0.1672 - val_loss: 0.1632
Epoch 8/100
47/47 [==============================] - 8s 169ms/step - loss: 0.1654 - val_loss: 0.1639
Epoch 9/100
47/47 [==============================] - 8s 164ms/step - loss: 0.1624 - val_loss: 0.1659
Epoch 10/100
47/47 [==============================] - 8s 166ms/step - loss: 0.1608 - val_l

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 84889), started 0:22:04 ago. (Use '!kill 84889' to kill it.)

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.3925439708050127
Val 0.3548004035887573
Test 0.1151007308580434


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.632534355209208, pvalue=0.0)
Val SpearmanrResult(correlation=0.6271331384065945, pvalue=1e-323)
Test SpearmanrResult(correlation=0.3709522531823497, pvalue=2.3789080298989024e-97)


### Freeze Conv 1, 2, Train Conv 3 from Scratch

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":2, 
                   "conv2":2, 
                   "conv3":0}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)         

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate_),  # https://stackoverflow.com/questions/59737875/keras-change-learning-rate
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=patience_)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=num_epochs_, 
                    batch_size=batch_size_,
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
47/47 [==============================] - 18s 354ms/step - loss: 2.0825 - val_loss: 1.2817
Epoch 2/100
47/47 [==============================] - 15s 324ms/step - loss: 0.7604 - val_loss: 0.6064
Epoch 3/100
47/47 [==============================] - 15s 318ms/step - loss: 0.3668 - val_loss: 0.4070
Epoch 4/100
47/47 [==============================] - 16s 338ms/step - loss: 0.2604 - val_loss: 0.3275
Epoch 5/100
47/47 [==============================] - 17s 363ms/step - loss: 0.1995 - val_loss: 0.2932
Epoch 6/100
47/47 [==============================] - 16s 342ms/step - loss: 0.1716 - val_loss: 0.3021
Epoch 7/100
47/47 [==============================] - 17s 360ms/step - loss: 0.1554 - val_loss: 0.2874
Epoch 8/100
47/47 [==============================] - 16s 331ms/step - loss: 0.1398 - val_loss: 0.3029
Epoch 9/100
47/47 [==============================] - 16s 347ms/step - loss: 0.1327 - val_loss: 0.2620
Epoch 10/100
47/47 [==============================] - 16s 334ms/step - loss: 0.126

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 84889), started 0:48:16 ago. (Use '!kill 84889' to kill it.)

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.9843885251101309
Val 0.25964817665242124
Test 0.06539060154501353


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.9964356703417023, pvalue=0.0)
Val SpearmanrResult(correlation=0.5167929897960384, pvalue=4.032463511650697e-202)
Test SpearmanrResult(correlation=0.26628072723330126, pvalue=2.7265864197887642e-49)


### Freeze Conv 1, 2, Train Conv 3 from Init

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":2, 
                   "conv2":2, 
                   "conv3":1}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)         

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate_),  # https://stackoverflow.com/questions/59737875/keras-change-learning-rate
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=patience_)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=num_epochs_, 
                    batch_size=batch_size_,
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
47/47 [==============================] - 18s 367ms/step - loss: 0.2211 - val_loss: 0.3036
Epoch 2/100
47/47 [==============================] - 18s 386ms/step - loss: 0.1322 - val_loss: 0.2434
Epoch 3/100
47/47 [==============================] - 15s 320ms/step - loss: 0.0485 - val_loss: 0.2328
Epoch 4/100
47/47 [==============================] - 15s 321ms/step - loss: 0.0312 - val_loss: 0.2019
Epoch 5/100
47/47 [==============================] - 16s 343ms/step - loss: 0.0239 - val_loss: 0.1956
Epoch 6/100
47/47 [==============================] - 16s 349ms/step - loss: 0.0209 - val_loss: 0.2085
Epoch 7/100
47/47 [==============================] - 17s 358ms/step - loss: 0.0194 - val_loss: 0.2032
Epoch 8/100
47/47 [==============================] - 16s 336ms/step - loss: 0.0168 - val_loss: 0.1913
Epoch 9/100
47/47 [==============================] - 17s 356ms/step - loss: 0.0153 - val_loss: 0.1938
Epoch 10/100
47/47 [==============================] - 16s 333ms/step - loss: 0.014

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 84889), started 1:02:01 ago. (Use '!kill 84889' to kill it.)

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.9790875958721533
Val 0.27465737787693567
Test 0.15724966579604183


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.9957482582995867, pvalue=0.0)
Val SpearmanrResult(correlation=0.5213717666673563, pvalue=2.609940248691938e-206)
Test SpearmanrResult(correlation=0.37349594525195046, pvalue=9.11073406747282e-99)


---
### Spearman Results

0 for init random weights & train  
1 for load pre-trained weights and train  
2 for load pre-trained weights and freeze  

First go:
000: val = 0.47753801170363896, test = 0.2866754020651326  
111: val = 0.47855750422978954, test = 0.279400514316224  
222: val = 0.5858565342434344, test = 0.3741217612323338  
220: val = 0.41782505305420903, test = 0.23175390117095385  
221: val = 0.4622724735247824, test = 0.26885707718958146  
2220: val = 0.5984249204802108, test = 0.32715495828441904

Second go:
000: val = 0.44123205768583007, test = 0.17504952705223212  
111: val = 0.4053507361644881, test = 0.2592776174804178  
222: val = 0.5290568130486669, test = 0.3636127380447914  
220: val = 0.392220547475502, test = 0.24043305401824872 
221: val = 0.40291026906804733, test = 0.22825701637010898
2220: val = 0.5988624237048076, test = 0.3301200533685432

### Train only output layer

In [ ]:
def get_model(treatment_dict):  # initializes model architecture
    mdl = Sequential()

    conv1_train = treatment_dict["conv1"] != 2  # bool
    mdl.add(Conv1D(120, 5, activation='relu', input_shape=(145, 4), name="1DConv_1", trainable=conv1_train))
    mdl.add(BatchNormalization(name="batchNorm1", trainable=conv1_train))
    mdl.add(Dropout(0.1, name="drop1"))

    conv2_train = treatment_dict["conv2"] != 2  # bool 
    mdl.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=conv2_train))
    mdl.add(BatchNormalization(name="batchNorm2", trainable=conv2_train))
    mdl.add(Dropout(0.1, name="drop2"))

    conv3_train = treatment_dict["conv3"] != 2  # bool 
    mdl.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=conv3_train))
    mdl.add(BatchNormalization(name="batchNorm3", trainable=conv3_train))
    mdl.add(Dropout(0.1, name="drop3"))

    mdl.add(Flatten(name="flat"))    
    mdl.add(Dense(12, activation='linear', name="dense2", trainable=False))
    mdl.add(Dense(1, activation='linear', name="dense3"))
    
    return mdl

def set_weights(treatment_dict, mdl):  # sets appropriate model weights from pretrained
    layers_to_set = [i for i in range(11)]
    
    for i in layers_to_set:
        print(mdl.layers[i])
        pretrained_layer_weights = pretrained_model.layers[i].get_weights()
        mdl.layers[i].set_weights(pretrained_layer_weights)
    
    return mdl

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":2, 
                   "conv2":2, 
                   "conv3":2}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)         

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate_),  # https://stackoverflow.com/questions/59737875/keras-change-learning-rate
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=patience_)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=num_epochs_, 
                    batch_size=batch_size_,
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
47/47 [==============================] - 12s 226ms/step - loss: 0.3706 - val_loss: 0.2286
Epoch 2/100
47/47 [==============================] - 9s 200ms/step - loss: 0.2532 - val_loss: 0.2121
Epoch 3/100
47/47 [==============================] - 10s 204ms/step - loss: 0.2363 - val_loss: 0.2047
Epoch 4/100
47/47 [==============================] - 9s 196ms/step - loss: 0.2280 - val_loss: 0.1977
Epoch 5/100
47/47 [==============================] - 9s 184ms/step - loss: 0.2231 - val_loss: 0.1921
Epoch 6/100
47/47 [==============================] - 9s 198ms/step - loss: 0.2176 - val_loss: 0.1879
Epoch 7/100
47/47 [==============================] - 9s 200ms/step - loss: 0.2151 - val_loss: 0.1846
Epoch 8/100
47/47 [==============================] - 8s 178ms/step - loss: 0.2133 - val_loss: 0.1824
Epoch 9/100
47/47 [==============================] - 9s 189ms/step - loss: 0.2088 - val_loss: 0.1806
Epoch 10/100
47/47 [==============================] - 9s 186ms/step - loss: 0.2106 - val_

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 84889), started 1:14:11 ago. (Use '!kill 84889' to kill it.)

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.25886857797184726
Val 0.31608893826491136
Test 0.0437756739890256


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.5124960925179921, pvalue=0.0)
Val SpearmanrResult(correlation=0.5998885838414266, pvalue=2.983051998914243e-289)
Test SpearmanrResult(correlation=0.32616550557136814, pvalue=2.0231700651400805e-74)
